## Work
1. 請比較使用 l1, l1_l2 及不同比例下的訓練結果

In [1]:
import os
import keras
import itertools

# Disable GPU
os.environ["CUDA_VISIBLE_DEVICES"] = ""

Using TensorFlow backend.


In [2]:
train, test = keras.datasets.cifar10.load_data()

In [3]:
## 資料前處理
def preproc_x(x, flatten=True):
    x = x / 255.
    if flatten:
        x = x.reshape((len(x), -1))
    return x

def preproc_y(y, num_classes=10):
    if y.shape[-1] == 1:
        y = keras.utils.to_categorical(y, num_classes)
    return y    

In [4]:
x_train, y_train = train
x_test, y_test = test

# Preproc the inputs
x_train = preproc_x(x_train)
x_test = preproc_x(x_test)

# Preprc the outputs
y_train = preproc_y(y_train)
y_test = preproc_y(y_test)

In [5]:
from keras.regularizers import l1_l2

# kernel_regularizer
"""
建立神經網路
"""
def build_mlp(input_shape, output_units=10, num_neurons=[512, 256, 128], l1_ratio=0.0, l2_ratio=0.0):
    input_layer = keras.layers.Input(input_shape)
    
    for i, n_units in enumerate(num_neurons):
        if i == 0:
            x = keras.layers.Dense(units=n_units, 
                                   activation="relu", 
                                   name="hidden_layer"+str(i+1), 
                                   kernel_regularizer=l1_l2(l1=l1_ratio, l2=l2_ratio))(input_layer)
        else:
            # x = keras.layers.Dense(units=n_units, activation="relu", name="hidden_layer"+str(i+1))(x)
            x = keras.layers.Dense(units=n_units, 
                                   activation="relu", 
                                   name="hidden_layer"+str(i+1),
                                   kernel_regularizer=l1_l2(l1=l1_ratio, l2=l2_ratio))(x)
    
    out = keras.layers.Dense(units=output_units, activation="softmax", name="output")(x)
    
    model = keras.models.Model(inputs=[input_layer], outputs=[out])
    return model

In [6]:
"""Code Here
設定超參數
"""

## 超參數設定
LEARNING_RATE = 1e-3
EPOCHS = 10
BATCH_SIZE = 256
MOMENTUM = 0.95
L1_EXP = [1e-2, 1e-4, 1e-8, 1e-12, 0.0]
L2_EXP = [1e-2, 1e-4, 1e-8, 1e-12, 0.0]

In [7]:
results = {}
"""Code Here
撰寫你的訓練流程並將結果用 dictionary 紀錄
"""

results = {}
for l1r, l2r in itertools.product(L1_EXP, L2_EXP):
    keras.backend.clear_session() # 把舊的 Graph 清掉
    print("Experiment with L1 = %.6f, L2 = %.6f" % (l1r, l2r))
    #model = build_mlp(input_shape=x_train.shape[1:])
    model = build_mlp(input_shape=x_train.shape[1:], l1_ratio=l1r, l2_ratio=l2r)
    model.summary()
    #optimizer = keras.optimizers.SGD(lr=lr, nesterov=True, momentum=momentum)
    optimizer = keras.optimizers.SGD(lr=LEARNING_RATE, nesterov=True, momentum=MOMENTUM)
    model.compile(loss="categorical_crossentropy", metrics=["accuracy"], optimizer=optimizer)

    model.fit(x_train, y_train, 
              epochs=EPOCHS, 
              batch_size=BATCH_SIZE, 
              validation_data=(x_test, y_test), 
              shuffle=True)
    
    # Collect results
    train_loss = model.history.history["loss"]
    valid_loss = model.history.history["val_loss"]
    train_acc = model.history.history["accuracy"]
    valid_acc = model.history.history["val_accuracy"]
    
    exp_name_tag = "exp-l1-%s-l2-%s" % (str(l1r), str(l2r))
    results[exp_name_tag] = {'train-loss': train_loss,
                             'valid-loss': valid_loss,
                             'train-acc': train_acc,
                             'valid-acc': valid_acc}

Experiment with L1 = 0.010000, L2 = 0.010000
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 3072)              0         
_________________________________________________________________
hidden_layer1 (Dense)        (None, 512)               1573376   
_________________________________________________________________
hidden_layer2 (Dense)        (None, 256)               131328    
_________________________________________________________________
hidden_layer3 (Dense)        (None, 128)               32896     
_________________________________________________________________
output (Dense)               (None, 10)                1290      
Total params: 1,738,890
Trainable params: 1,738,890
Non-trainable params: 0
_________________________________________________________________
Train on 50000 samples, validate on 10000 samples
Epoch 1/10
50000/500

Train on 50000 samples, validate on 10000 samples
Epoch 1/10
50000/50000 [==============================] - 14s 285us/step - loss: 198.8021 - accuracy: 0.2273 - val_loss: 41.2195 - val_accuracy: 0.2149accuracy: 
Epoch 2/10
50000/50000 [==============================] - 13s 260us/step - loss: 19.1402 - accuracy: 0.1163 - val_loss: 7.2808 - val_accuracy: 0.1000
Epoch 3/10
50000/50000 [==============================] - 13s 262us/step - loss: 4.1132 - accuracy: 0.0981 - val_loss: 2.6486 - val_accuracy: 0.1000
Epoch 4/10
50000/50000 [==============================] - 14s 282us/step - loss: 2.4860 - accuracy: 0.0966 - val_loss: 2.4626 - val_accuracy: 0.1000
Epoch 5/10
50000/50000 [==============================] - 14s 283us/step - loss: 2.4626 - accuracy: 0.0969 - val_loss: 2.4625 - val_accuracy: 0.1000
Epoch 6/10
50000/50000 [==============================] - 14s 275us/step - loss: 2.4626 - accuracy: 0.0998 - val_loss: 2.4627 - val_accuracy: 0.1000
Epoch 7/10
50000/50000 [==================

50000/50000 [==============================] - 17s 346us/step - loss: 5.8512 - accuracy: 0.3611 - val_loss: 5.7667 - val_accuracy: 0.3814
Epoch 3/10
50000/50000 [==============================] - 15s 292us/step - loss: 5.6968 - accuracy: 0.3966 - val_loss: 5.6302 - val_accuracy: 0.4089
Epoch 4/10
50000/50000 [==============================] - 13s 258us/step - loss: 5.5711 - accuracy: 0.4155 - val_loss: 5.5137 - val_accuracy: 0.4237
Epoch 5/10
50000/50000 [==============================] - 14s 277us/step - loss: 5.4570 - accuracy: 0.4327 - val_loss: 5.4259 - val_accuracy: 0.4240
Epoch 6/10
50000/50000 [==============================] - 14s 282us/step - loss: 5.3507 - accuracy: 0.4453 - val_loss: 5.3282 - val_accuracy: 0.4369
Epoch 7/10
50000/50000 [==============================] - 14s 274us/step - loss: 5.2498 - accuracy: 0.4566 - val_loss: 5.2168 - val_accuracy: 0.4526
Epoch 8/10
50000/50000 [==============================] - 13s 265us/step - loss: 5.1558 - accuracy: 0.4656 - val_loss

50000/50000 [==============================] - 11s 228us/step - loss: 5.4512 - accuracy: 0.4125 - val_loss: 5.3962 - val_accuracy: 0.4236
Epoch 5/10
50000/50000 [==============================] - 12s 234us/step - loss: 5.3431 - accuracy: 0.4306 - val_loss: 5.2967 - val_accuracy: 0.4319
Epoch 6/10
50000/50000 [==============================] - 12s 236us/step - loss: 5.2437 - accuracy: 0.4447 - val_loss: 5.2061 - val_accuracy: 0.4376
Epoch 7/10
50000/50000 [==============================] - 12s 245us/step - loss: 5.1493 - accuracy: 0.4554 - val_loss: 5.1189 - val_accuracy: 0.4514
Epoch 8/10
50000/50000 [==============================] - 12s 242us/step - loss: 5.0601 - accuracy: 0.4650 - val_loss: 5.0375 - val_accuracy: 0.4486
Epoch 9/10
50000/50000 [==============================] - 13s 254us/step - loss: 4.9743 - accuracy: 0.4734 - val_loss: 4.9537 - val_accuracy: 0.4664
Epoch 10/10
50000/50000 [==============================] - 12s 250us/step - loss: 4.8890 - accuracy: 0.4821 - val_los

50000/50000 [==============================] - 14s 274us/step - loss: 1.5805 - accuracy: 0.4473 - val_loss: 1.5705 - val_accuracy: 0.4557
Epoch 7/10
50000/50000 [==============================] - 13s 268us/step - loss: 1.5459 - accuracy: 0.4587 - val_loss: 1.5440 - val_accuracy: 0.4527
Epoch 8/10
50000/50000 [==============================] - 13s 269us/step - loss: 1.5157 - accuracy: 0.4692 - val_loss: 1.5285 - val_accuracy: 0.4555
Epoch 9/10
50000/50000 [==============================] - 14s 272us/step - loss: 1.4894 - accuracy: 0.4765 - val_loss: 1.5072 - val_accuracy: 0.4633
Epoch 10/10
50000/50000 [==============================] - 14s 278us/step - loss: 1.4645 - accuracy: 0.4865 - val_loss: 1.4811 - val_accuracy: 0.4767
Experiment with L1 = 0.000000, L2 = 0.000000
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 3072)              0         
_____

50000/50000 [==============================] - 13s 256us/step - loss: 6.0919 - accuracy: 0.4243 - val_loss: 5.7599 - val_accuracy: 0.4235
Epoch 9/10
50000/50000 [==============================] - 12s 249us/step - loss: 5.4564 - accuracy: 0.4305 - val_loss: 5.1791 - val_accuracy: 0.4233
Epoch 10/10
50000/50000 [==============================] - 13s 250us/step - loss: 4.9126 - accuracy: 0.4358 - val_loss: 4.6703 - val_accuracy: 0.4391
Experiment with L1 = 0.000000, L2 = 0.000100
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 3072)              0         
_________________________________________________________________
hidden_layer1 (Dense)        (None, 512)               1573376   
_________________________________________________________________
hidden_layer2 (Dense)        (None, 256)               131328    
_______________________________________

ResourceExhaustedError:  OOM when allocating tensor with shape[3072,512] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator cpu
	 [[node hidden_layer1/weight_regularizer/Square (defined at C:\Users\liv\anaconda3\envs\NLP_course\lib\site-packages\keras\backend\tensorflow_backend.py:3009) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.
 [Op:__inference_keras_scratch_graph_232169]

Function call stack:
keras_scratch_graph


In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
"""Code Here
將結果繪出
"""

import matplotlib.cm as mplcm
import matplotlib.colors as colors
NUM_COLORS = 25

cm = plt.get_cmap('gist_rainbow')
cNorm  = colors.Normalize(vmin=0, vmax=NUM_COLORS-1)
scalarMap = mplcm.ScalarMappable(norm=cNorm, cmap=cm)
color_bar = [scalarMap.to_rgba(i) for i in range(NUM_COLORS)]

plt.figure(figsize=(8,6))
for i, cond in enumerate(results.keys()):
    plt.plot(range(len(results[cond]['train-loss'])),results[cond]['train-loss'], '-', label=cond, color=color_bar[i])
    plt.plot(range(len(results[cond]['valid-loss'])),results[cond]['valid-loss'], '--', label=cond, color=color_bar[i])
plt.title("Loss")
plt.ylim([0, 5])
plt.legend()
plt.show()

plt.figure(figsize=(8,6))
for i, cond in enumerate(results.keys()):
    plt.plot(range(len(results[cond]['train-acc'])),results[cond]['train-acc'], '-', label=cond, color=color_bar[i])
    plt.plot(range(len(results[cond]['valid-acc'])),results[cond]['valid-acc'], '--', label=cond, color=color_bar[i])
plt.title("Accuracy")
plt.legend()
plt.show()